In [ ]:
import backtest
import numpy as np
import pandas as pd

In [ ]:
DATA_PATH = "./data/"
dat = pd.read_parquet(DATA_PATH + "eurusd.parquet")
dat.shape, dat.columns

In [ ]:
opt_trades = {}
dd_constraints = np.arange(10,100,10)
for dd_bps in dd_constraints:
    opt_trades[dd_bps] = backtest.target_optimal(df_price = dat.close,
                                                 fee_bps = 3,
                                                 dd_bps = dd_bps)
    
    print(f"Done optimizing labels with dd_bps constraint = {dd_bps}.")

In [ ]:
stats = {} # trading statistics
for dd_bps in dd_constraints:
    trades_in = dat.copy()
    trades_in['S'] = 0
    trades_in['L'] = 0
    trades_in['C'] = 0
    trades_in.loc[opt_trades[dd_bps] == 0, 'S'] = 1
    trades_in.loc[opt_trades[dd_bps] == 1, 'L'] = 1
    trades_in.loc[opt_trades[dd_bps] == 2, 'C'] = 1
    
    b = backtest.BackTest(data = trades_in,
                          decimal_pip = 5,
                          threshold = 0.5,
                          fee_bps = 3)
    stats[dd_bps] = b.trade_stats()